In [ ]:
import pandas as pd
import seaborn
from matplotlib import pyplot as plt
import os

In [ ]:
data_dir = "./data/"
data = []
for file in os.listdir(data_dir):
    data_raw = pd.read_excel(data_dir + file, skiprows=4)
    data_raw.columns = [column.lower().replace("\n", "").replace(" ", "_").replace("-", "_").strip() for column in data_raw.columns]
    data.append(data_raw)
    
data = pd.concat(data)
print(data.columns)

In [ ]:
def get_region(region):
    if region == 1:
        return "Manhattan"
    elif region == 2:
        return "Bronx"
    elif region == 3:
        return "Brooklyn"
    elif region == 4:
        return "Queens"
    elif region == 5:
        return "Staten Island"


data.insert(1, "district", [get_region(a) for a in data.region])
data

In [ ]:
data.info()

In [ ]:
free_houses = data[(data.total_units > 0) & (data.land_square_feet > 0)]
free_houses['quantile'] = pd.qcut(free_houses['saleprice'], 10, labels=False, duplicates='drop')
free_houses = free_houses[(free_houses['quantile'] > 0) & (free_houses['quantile'] < 10)].drop(columns="quantile")
free_houses['year_quarter'] = free_houses.sale_date.apply(lambda x: f"{x.year}-{x.quarter}")
free_houses

In [ ]:
free_houses['price_per_lsf'] = free_houses.saleprice / free_houses.land_square_feet
free_houses

In [ ]:
houses_for_plot = free_houses.drop(
    columns=["sale_date", "address", "zip_code", "block", "lot", "ease_ment", "building_class_at_present","residential_units", "commercial_units", "total_units", "year_built"])
houses_for_plot

In [ ]:
houses_by_year = houses_for_plot.groupby(["year_quarter"]).mean()
houses_by_year

In [ ]:
houses_by_year.price_per_lsf.plot(kind="bar")

In [ ]:
houses_by_region = houses_for_plot.groupby(['district', 'year_quarter'], as_index=False).mean()
houses_by_region

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax = seaborn.lineplot(data=houses_by_region, x=houses_by_region.year_quarter, y=houses_by_region.price_per_lsf,hue=houses_by_region.district)
seaborn.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))